# Verify Presence of Established Primer Pairs

Given the `nt` data set and some published primer sets, the current script verifies if they are in the set of kmers identified by PriSeT.

In [2]:
import collections

'''
name: short string ID
name_fwd: string ID for forward primer
name_rev: string ID for reverse primer
seq_fwd: one-letter encoded DNA sequence of forward primer
seq_rev: one-letter encoded DNA sequence of reverse primer
target_region: genome location for which primer was originally designed
target_group: organism group for which primer was originally designed
product_length: expected amplicon length (without primer sequences) 
ref_pub: original publication referring to this primer pair
'''
primer_pair = collections.namedtuple('primer_pair', \
                'name name_fwd name_rev seq_fwd seq_rev target_region target_group product_length ref_pub')
# Some known primers
primer_pairs = [ \
    primer_pair._make(["DIV4", "DIV4_fw", "DIV4_rv", "GCGGTAATTCCAGCTCCAATAG", "CTCTGACAATGGAATACGAATA", None, "Diatoms", 329, "https://www.ncbi.nlm.nih.gov/pubmed/26052741"]),\
    primer_pair._make(["EUK14", "F-566cw-bio", "R-1200", "CAGCAGCCGCGGTAATTCC", "CCCGTGTTGAGTCAAATTAAGC", "18S", "Eukaryotes", None, None]), \
    primer_pair._make(["EUK15", "TAReuk454FWD1", "TAReukREV3", "CCAGCASCYGCGGTAATTCC", "ACTTTCGTTCTTGATYRA", "18S", "Eukaryotes", None, "http://www.biomarks.eu/sites/default/files/pdf-refs/Stoeck%20et%20al%202010.pdf"]), \
    primer_pair._make(["UNIV", "926mod", "1392mod", "AAACTYRAAGWGRCGG", "AAAGTCTCGTGWGTRC", "16S", "Universal", None, None]), \
    primer_pair._make(["23S", "A23SrVF1", "A23SrVR1", "GGACARAAAGACCCTATG", "AGATCAGCCTGTTATCC", None, None, 400, "https://peerj.com/articles/2115/"]), \
    primer_pair._make(["COI", "LCO1490", "HC02198", "GGTCAACAAATCATAAAGATATTGG", "TAAACTTCAGGGTGACCAAAAAATCA", "23S", None, 230, "https://pdfs.semanticscholar.org/943d/38b9d96f8222e883604822bcafb7930ca6da.pdf"]), \
    primer_pair._make(["nSSU", "SU_FO4 nSSU", "SSU_R22 nSSU", "GCTTGTCTCAAAGATTAAGCC", "GCCTGCTGCCTTCCTTGGA", "18S", "multiple metazoan phyla", None, "https://www.nature.com/articles/ncomms1095"]), \
    primer_pair._make(["V9", "V9_18S_fw", "V9_18S_rv", "GTACACACCGCCCGTC", "TGATCCTTCTGCAGGTTCACCTAC", "18S", "zooplankton", None, None]) \
    ]
one_letter_codes = {'B': 'CGT', 'D': 'AGT', 'H': 'ACT', 'K': 'GT', 'M': 'AC', 'N': 'ACGT', 'R': 'AG', 'S': 'CG', \
                    'V': 'ACG', 'W': 'AT', 'Y': 'CT'}

### Build Settings and Index Retrieval

In [60]:
import os

# first LIMIT fasta entries from nt dataset, set LIMIT <=0 to process all accessions
LIMIT = 25000
LIMIT_STR = str(int(LIMIT/1000)) + 'K'

volume = "/Volumes/plastic_data" # "/path/to/volume"
taxid = 'nt_{}'.format(LIMIT_STR) if (LIMIT > 0) else 'nt'  # cellular, non-human organisms
priset_build = os.path.expanduser('~/git/PriSeT_git/build')
priset_bin = os.path.join(priset_build, 'priset_180719')
genmap_bin = os.path.expanduser('~/git/genmap-build/bin/genmap')
src_dir = os.path.expanduser('~/git/PriSeT_git/PriSeT/src/priset.cpp')
lib_dir = '{}/priset/library/{}'.format(volume, taxid)
tmp_dir = '{}/priset/tmp'.format(volume)
work_dir = '{}/priset/work/{}'.format(volume, taxid)
idx_dir = os.path.join(work_dir, 'index')
result_dir = os.path.join(work_dir, 'table')
# taxid,acc1,acc2
acc_file = os.path.join(lib_dir, "{}.acc".format(taxid))
# acc id to accession number
id_file = os.path.join(lib_dir, "{}.id".format(taxid))
# taxid, p_taxid, is_species
tax_file = os.path.join(lib_dir, "{}.tax".format(taxid))
# url to nt dataset
url_ref = 'ftp://ftp.ncbi.nlm.nih.gov/blast/db/FASTA/nt.gz'
# archive file name
ref_arx = os.path.join(tmp_dir, os.path.basename(url_ref)[:-3]) # remove '.gz'
# future unzipped reference file
ref_file = os.path.join(lib_dir, '{}.fasta'.format(taxid)) 


## Download and Build Library

### Download nt Dataset

In [12]:
import subprocess

print(subprocess.run(['wget', URL_REF, '-N', '-P', tmp_dir], stdout=subprocess.PIPE))
print(subprocess.check_output('gunzip {}'.format(ref_arx), shell=True))

if os.path.isfile(ref_arx) and os.path.exists(os.path.dirname(ref_file)):
    print(subprocess.check_output('mv {} {}'.format(ref_arx, ref_file), shell=True))
else:
    print("ref_arx is file: ", os.path.isfile(ref_arx), ", target path exists: ", os.path.exists(os.path.dirname(ref_file)))

/Volumes/plastic_data/priset/tmp/nt.gz
/Volumes/plastic_data/priset/tmp/nt
/Volumes/plastic_data/priset/library/nt/nt.fasta
b''


### Download Taxonomy DB

The provided archive provides tables from which we extract the hierarchical information of taxa and which taxon has accessions assigned to it. 

In [46]:
import subprocess

# download taxonomy DB files
url_tax = 'ftp://ftp.ncbi.nih.gov/pub/taxonomy/new_taxdump/new_taxdump.tar.gz'

cmd = ' '.join(['wget', url_tax, '-P', tmp_dir, '-N'])

proc = subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE, shell=True)
stdout_value, stderr_value = proc.communicate()
print("stdout: ", stdout_value.decode('utf-8'))
#print("stderr: ", stderr_value.decode('utf-8'))
    
# unpack archive 
print(subprocess.check_output('tar xzf {} -C {}'.format(os.path.join(tmp_dir, os.path.basename(url_tax)), tmp_dir), stderr=subprocess.STDOUT, shell=True))


stdout:  
stderr:  --2019-07-16 17:48:43--  ftp://ftp.ncbi.nih.gov/pub/taxonomy/new_taxdump/new_taxdump.tar.gz
           => >>/Volumes/plastic_data/priset/tmp/.listing<<
Aufl"osen des Hostnamens ftp.ncbi.nih.gov... 130.14.250.10, 2607:f220:41e:250::10
Verbindungsaufbau zu ftp.ncbi.nih.gov|130.14.250.10|:21 ... verbunden.
Anmelden als anonymous ... Angemeldet!
==> SYST ... fertig.    ==> PWD ... fertig.
==> TYPE I ... fertig.  ==> CWD (1) /pub/taxonomy/new_taxdump ... fertig.
==> PASV ... fertig.    ==> LIST ... fertig.

     0K                                                         213K=0.002s

2019-07-16 17:48:45 (213 KB/s) - >>/Volumes/plastic_data/priset/tmp/.listing<< gespeichert [539]

>>/Volumes/plastic_data/priset/tmp/.listing<< gel"oscht.
--2019-07-16 17:48:45--  ftp://ftp.ncbi.nih.gov/pub/taxonomy/new_taxdump/new_taxdump.tar.gz
           => >>/Volumes/plastic_data/priset/tmp/new_taxdump.tar.gz<<
==> CWD nicht erforderlich.
==> PASV ... fertig.    ==> RETR new_taxdump.tar.gz

b''


### Download taxid2accession File

In [14]:
import subprocess

url_acc2tax = 'ftp://ftp.ncbi.nih.gov/pub/taxonomy/accession2taxid/nucl_gb.accession2taxid.gz'
cmd = ' '.join(['wget', url_acc2tax, '-P', tmp_dir, '-N'])
'''
proc = subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE, shell=True)
stdout_value, stderr_value = proc.communicate()
print("stdout: ", stdout_value.decode('utf-8'))
print("stderr: ", stderr_value.decode('utf-8'))
'''    
# unpack archive 
print(subprocess.check_output('gunzip {}'.format(os.path.join(tmp_dir, os.path.basename(url_acc2tax))), stderr=subprocess.STDOUT, shell=True))


b''


### Create nt_sub.fasta from Complete Reference

In [72]:
nt_file = os.path.join(volume, 'priset/library/nt/nt.fasta')
with open(nt_file, 'r') as f_nt:  #, open(ref_file, 'w') as f_ref:
    line = f_nt.readline()
    acc_ctr = 0
    while line:
        if line.startswith('>'):
            #if acc_ctr > LIMIT:
            #   break
            acc_ctr += 1
        #f_ref.write(line)
        line = f_nt.readline()
print("STATUS: ", acc_ctr, " entries in nt") #written to ", f_ref)
            


STATUS:  53045220  entries in nt


### Create nt.id File from Reference

Built from `nt.fasta`, the file is a map between reference IDs (1-based) and accession numbers following the same order than in the `nt.fasta` file.

In [64]:
print(ref_file, id_file)
with open(ref_file, 'r') as f_ref, open(id_file, 'w') as f_id:
    print('here')
    line = f_ref.readline()
    acc_id = 1
    f_id.write("index,accession\n")
    while line:
        if line.startswith('>'):
            if LIMIT > 0 and acc_id > LIMIT:
                break
            acc_no = line.split(' ')[0][1:]
            f_id.write("{},{}\n".format(acc_id, acc_no))
            acc_id += 1
        line = f_ref.readline()

print("STATUS\twrote", acc_id, 'entries to', id_file)

/Volumes/plastic_data/priset/library/nt_25K/nt_25K.fasta /Volumes/plastic_data/priset/library/nt_25K/nt_25K.id
here
STATUS	wrote 25001 entries to /Volumes/plastic_data/priset/library/nt_25K/nt_25K.id


### Create nt.acc File from nucl_gb.accession2taxid

The source file is an accession to taxid map. Hence, we will accumulate accessions by taxid in main memory before writing back.

In [65]:
# load only subsetted accessions
subset = set()
print(id_file)
with open(id_file, 'r') as f_id:
    line = f_id.readline()
    line = f_id.readline()
    while line:
        subset.add(line.split(',')[1].strip())
        line = f_id.readline()
print('INFO\tThere are', len(subset), 'accessions')

# src format: accession accession.version taxid gi
acc2taxid_file = os.path.join(tmp_dir, os.path.basename(url_acc2tax[:-3]))
if os.path.isfile(acc2taxid_file) and os.path.exists(os.path.dirname(acc_file)):
    tax2acc = {}
    with open(acc2taxid_file, 'r') as f_src:
        line = f_src.readline()  # skip header line
        line = f_src.readline()
        while line:
            line = line.split('\t')
            acc, tax = line[1].strip(), line[2].strip()
            if acc in subset:
                acc_list = tax2acc.get(tax, "") + "," + acc
                tax2acc[tax] = acc_list
            line = f_src.readline()
    with open(acc_file, 'w') as f_dest:
        f_dest.write("taxid,acc1,acc2,...\n")
        for tax in sorted(list(tax2acc.keys())):
            f_dest.write("{},{}\n".format(tax, tax2acc[tax][1:]))  # truncate leading comma
        print("STATUS\ttaxids and accessions written to", acc_file)
else:
    print("ERROR\t nucl_gb.accession2taxid does not exists here: ", acc2taxid_file, " or destiny folder does not exists: ", )

/Volumes/plastic_data/priset/library/nt_25K/nt_25K.id
INFO	There are 1 accessions
STATUS	taxids and accessions written to /Volumes/plastic_data/priset/library/nt_25K/nt_25K.acc


### Create nt.tax File 

In [31]:
leaf_ranks = set(['species', 'varietas', 'forma', 'subspecies', 'subvarietas', 'subforma'])
nodes_file = os.path.join(tmp_dir, "nodes.dmp")  #taxid, parent taxid, rank, etc.
with open(nodes_file, 'r') as f_nodes, open(tax_file, 'w') as f_tax:
    line = f_nodes.readline()
    f_tax.write("taxid,p_taxid,is_species\n")
    while line:
        cells = line.split('|')
        if len(cells) < 4:
            print("Error: unexpected nodes.dmp format: ", line)
            sys.exit(0)
        taxid = int(cells[0].strip())
        p_taxid = int(cells[1].strip())
        is_species = 1 if (cells[2].strip() in leaf_ranks) else 0 
        f_tax.write("{},{},{}\n".format(taxid, p_taxid, is_species))
        line = f_nodes.readline()
    print("STATUS\ttaxonomy written to", tax_file)

STATUS	taxonomy written to /Volumes/plastic_data/priset/library/nt_100000/nt_100000.tax


## Build Index
The index has to be built only once. Since the current size of the `nt` data set is 250 GB and its index computation wouldn't fit into main memory, we need to set the algorithm flag to [`skew`](https://github.com/cpockrandt/genmap) and set an environmental parameter indicating a working directory with sufficient capacity. 
Since I got a `Killed: 9` signal when building on my Laptop with connected hard drive of 1.4 TB free memory, I outsourced index computation to `redwood` and followed the genmap [installation](https://github.com/cpockrandt/genmap) steps.


In [71]:
import timeit

large_tmp_dir = os.path.join(volume, 'tmp')
# export TMPDIR=/somewhere/else/with/more/space
print(os.system('export TMPDIR={}'.format(large_tmp_dir)))
start = timeit.default_timer()
print(genmap_bin)
print(ref_file)
print(idx_dir)
cmd = '{} index -F {} -I {} -A skew'.format(genmap_bin, ref_file, idx_dir)
print(cmd)
proc = subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE, shell=True, cwd=priset_build)
stdout_value, stderr_value = proc.communicate()
stop = timeit.default_timer()
print('Elapsed time: ', stop - start, ' seconds')
    
print("stdout: ", stdout_value.decode('utf-8'))
#print("stderr: ", stderr_value.decode('utf-8'))

0
/Users/troja/git/genmap-build/bin/genmap
/Volumes/plastic_data/priset/library/nt_25K/nt_25K.fasta
/Volumes/plastic_data/priset/work/nt_25K/index
/Users/troja/git/genmap-build/bin/genmap index -F /Volumes/plastic_data/priset/library/nt_25K/nt_25K.fasta -I /Volumes/plastic_data/priset/work/nt_25K/index -A skew
Elapsed time:  39.6813410060131  seconds
stdout:  The index will now be built. This can take some time (e.g., 2-3 hours with Skew7 for the human genome).
Create fwd Index ... done!
Create bwd Index ... done!
Index created successfully.



In [28]:
def unfold(sequence):
    sequences = [sequence]
    for code, decodes in one_letter_codes.items():
        tmp = []
        for sequence in sequences:
            queue = [sequence]
            done = []
            pos = -1
            while len(queue) > 0:
                curr = queue.pop()
                pos = curr.find(code)
                if pos == -1:
                    done.append(curr)
                else:
                    for decode in decodes:
                        curr_new = curr[:pos] + decode + curr[pos+1:]
                        queue.append(curr_new)
            tmp += done 
        sequences = tmp
    return sequences

#print(unfold("BAY"))

In [ ]:
# known primers that are also found by PriSeT have flag set to one
primers_found = {}
# {primer_name: set_unfolded sequences}
primers_unfolded = {}
min_K = -1, max_K = -1
for primer_pair in primer_pairs:
    primers_found[primer_pair.name_fwd] = 0
    primers_found[primer_pair.name_rev] = 0
    primers_unfolded[primer_pair.name_fwd] = set(unfold(primer_pair.seq_fwd))
    primers_unfolded[primer_pair.name_rev] = set(unfold(primer_pair.seq_rev))
    min_aux = min(len(primer_pair.seq_fwd, primer_pair.seq_rev))
    max_aux = max(len(primer_pair.seq_fwd, primer_pair.seq_rev))
    min_K = (min_aux < min_K) ? min_aux : min_K
    max_K = (max_aux > max_K) ? max_aux : max_K
        
# map once for each unique primer length
for K in range(min_K, min_K+1):  # max_K + 1):
    # always skip index computation!
    cmd = [priset_bin, '-l', lib_dir, '-w', work_dir, '-K', str(K), '-s']
    
    # get primer result file
    primer_file = os.path.join(result_dir, 'primer_info.csv')
    primer_found = set()
    with open(primer_file, 'r') as pf:
        line = pf.readline()
        line = pf.readline()
        while line:
            cells = line.split(',')
            if (len(cells) < 2):
                print("Error: unusual primer info file format ", line)
            else:
                primer_found.add(line.s[1])
                
    for primer_pair in primer_pairs:
        for name, seq in zip([primer_pair.name_fwd, primer_pair.name_rev], [primer_pair.seq_fwd, primer_pair.seq_rev]):
            if len(seq) == K:
                for primer_unfolded in primers_unfolded[seq]:
                    if primer_unfolded in primers_found:
                        primers_intersect[name] = 1
print("primer\tfwd found\trev found\n-----------------------------------")
for primer_pair in primer_pairs:
    print(primer_pair.name, '\t', primers_intersect[primer_pair.name_fwd], '\t', primers_intersect[primer_pair.name_rev])
    